In [1]:
!python3.6 -m pip install  testresources
!python3.6  -m pip install tensorflow-gpu==2.0.0
!python3.6 -m pip install --upgrade pillow
!python3.6 -m pip install keras==2.3.1
!python3.6 -m pip install  'h5py==2.10.0' --force-reinstall
!python3.6 -m pip install scikit-image
!python3.6  -m pip install tensorflow==2.0.0
!python3.6  -m pip install opencv-contrib-python-headless==4.5.3.56



Requirement already up-to-date: pillow in /home/ceoebenezer/.local/lib/python3.6/site-packages (8.4.0)
  Using cached h5py-2.10.0-cp36-cp36m-manylinux1_x86_64.whl (2.9 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0


In [2]:
!python3.6 -m pip install pandas

In [3]:
from __future__ import absolute_import
from __future__  import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py


In [4]:
import os

In [5]:
from mamonfight22 import *

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import numpy as np
from skimage.transform import resize
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")
np.random.seed(1234)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1372843656967686873
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10550163840166939909
physical_device_desc: "device: XLA_CPU device"
]


In [ ]:
def mamon_videoFightModel2(tf,wight='mamonbest947oscombo.h5'):
    layers = tf.keras.layers
    models = tf.keras.models
    losses = tf.keras.losses
    optimizers = tf.keras.optimizers
    metrics = tf.keras.metrics
    num_classes = 2
    cnn = models.Sequential()
    #cnn.add(base_model)

    input_shapes=(160,160,3)
    np.random.seed(1234)
    vg19 = tf.keras.applications.vgg19.VGG19
    base_model = vg19(include_top=False,weights='imagenet',input_shape=(160, 160,3))
    # Freeze the layers except the last 4 layers
    #for layer in base_model.layers:
    #    layer.trainable = False

    cnn = models.Sequential()
    cnn.add(base_model)
    cnn.add(layers.Flatten())
    model = models.Sequential()

    model.add(layers.TimeDistributed(cnn,  input_shape=(30, 160, 160, 3)))
    model.add(layers.LSTM(30 , return_sequences= True))

    model.add(layers.TimeDistributed(layers.Dense(90)))
    model.add(layers.Dropout(0.1))

    model.add(layers.GlobalAveragePooling1D())

    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(num_classes, activation="sigmoid"))

    adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.load_weights(wight)
    rms = optimizers.RMSprop()

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

    return model
model = mamon_videoFightModel2(tf)

In [22]:
#### import cv2
from skimage.transform import resize
import numpy as np
print("hellooo")
cap = cv2.VideoCapture('/edgar1/fight_detection/Real-Time-Violence-Detection-in-Video-/output_video.mp4')
i = 0
frames = np.zeros((30, 160, 160, 3), dtype=np.float)
old = []
j = 0

while True:
    ret, frame = cap.read()

    if not ret:
        print("no frame ****")
        break  # Break if there are no more frames
        

    font = cv2.FONT_HERSHEY_SIMPLEX

    if i > 29:
        print("checking ")
        ysdatav2 = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
        ysdatav2[0][:][:] = frames
        # Assuming you have a pred_fight function defined
        predaction = pred_fight(model, ysdatav2, acuracy=0.975)
        # Perform actions based on the prediction

        if predaction[0] == True:
            cv2.putText(frame,
                        'Violence Detected... Violence.. violence',
                        (50, 50),
                        font, 3,
                        (0, 255, 255),
                        2,
                        cv2.LINE_4)
            #cv2.imshow('video', frame)
            print('Violence detected here...')
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            vio = cv2.VideoWriter(f"./videos/output-{j}.avi", fourcc, 10.0, (fwidth, fheight))
            for frameinss in old:
                vio.write(frameinss)
            vio.release()

        i = 0
        j += 1
        frames = np.zeros((30, 160, 160, 3), dtype=np.float)
        old = []
    else:
        # Check if frame is not None before processing
        if frame is not None:
            frm = resize(frame, (160, 160, 3))
            old.append(frame)
            fshape = frame.shape
            fheight = fshape[0]
            fwidth = fshape[1]
            
            frm = np.expand_dims(frm, axis=0)
            if np.max(frm) > 1:
                frm = frm / 255.0
            frames[i][:] = frm

            i += 1

    #cv2.imshow('video', frame)

    #if cv2.waitKey(1) & 0xFF == ord('q'):
    #    break

#cap.release()
#cv2.destroyAllWindows()


hellooo
checking 
checking 
checking 
checking 
checking 
checking 
checking 
Violence detected here...
checking 
Violence detected here...
checking 
Violence detected here...
checking 
checking 
checking 
checking 
checking 
checking 
checking 
Violence detected here...
checking 
checking 
Violence detected here...
checking 
Violence detected here...
checking 
checking 
checking 
checking 
checking 
checking 
Violence detected here...
checking 
Violence detected here...
checking 
checking 
checking 
checking 
checking 
checking 
no frame ****


In [16]:
import time
millis = int(round(time.time() * 1000))
print("started at " , millis)
vid = video_mamonreader(cv2,'dvpalfight.dvd')
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)

started at  1706088586748
(30, 160, 160, 3)


UnboundLocalError: local variable 'frame' referenced before assignment

In [ ]:
datav = np.zeros((1, 30, 160, 160, 3), dtype=np.float)

In [ ]:
datav[0][:][:] = vid

In [ ]:
millis = int(round(time.time() * 1000))
print("started at " , millis)
print(pred_fight(model,datav,acuracy=0.6))
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)

In [ ]:
def video_mamonreader(cv2,filename):
    frames = np.zeros((30, 160, 160, 3), dtype=np.float)
    i=0
    print(frames.shape)
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval , frame = vc.read()
    else:
        rval = False
    frm = resize(frame,(160,160,3))
    frm = np.expand_dims(frm,axis=0)
    if(np.max(frm)>1):
        frm = frm/255.0
    frames[i][:] = frm
    i +=1
    print("reading video")
    while i < 30:
        rval, frame = vc.read()
        frm = resize(frame,(160,160,3))
        frm = np.expand_dims(frm,axis=0)
        if(np.max(frm)>1):
            frm = frm/255.0
        frames[i][:] = frm
        i +=1
    return frames

In [ ]:
novid = video_mamonreader(cv2,'nofight.mp4')

In [ ]:
nodatav = np.zeros((1, 40, 170, 170, 3), dtype=np.float)

In [ ]:
nodatav[0][:][:] = novid

In [ ]:
pred_fight(model,nodatav,acuracy=0.6)

In [ ]:
ysvid2 = video_mamonreader(cv2,'hdfight.mp4')

In [ ]:
ysdatav2 = np.zeros((1, 30, 160, 160, 3), dtype=np.float)

In [ ]:
ysdatav2[0][:][:] = ysvid2

In [ ]:
import time

millis = int(round(time.time() * 1000))
print("started at " , millis)
predaction = pred_fight(model,ysdatav2,acuracy=0.9)
print(predaction)
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)


In [ ]:
if predaction[0] == True:
    print('violence')

In [ ]:
novid3 = video_mamonreader(cv2,'golsss.mp4')

In [ ]:
nodatav3 = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
nodatav3[0][:][:] = novid3

In [ ]:
millis = int(round(time.time() * 1000))
print("started at " , millis)
print(pred_fight(model,nodatav3,acuracy=0.8))
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)


In [ ]:

novid4 = video_mamonreader(cv2,'dvpalfight.dvd')

In [ ]:
nodatav4 = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
nodatav4[0][:][:] = novid4

In [ ]:
millis = int(round(time.time() * 1000))
print("started at " , millis)
print(pred_fight(model,nodatav4,acuracy=0.9))
millis2 = int(round(time.time() * 1000))
print("time processing " , millis2 - millis)